# Pseudocódigo

* num_tacos = Semaforo(0)
* espacio_libre = Semaforo(espacio)
* tomando_canasta = Semaforo(1)


código taquero:

    repite infinitamente:
        1. taco = haz_taco()
        2. P(espacio_libre)
        3. P(tomando_canasta)
        4. pon_en_canasta(taco)
        5. V(tomando_canasta)
        6. V(num_tacos)

código cliente:

    repite infinitamente:
        1. P(num_tacos)
        2. P(tomando_canasta)
        3. taco = toma_de_canasta()
        4. V(tomando_canasta)
        5. V(espacio_libre)


# Implementación

In [1]:
import multiprocessing as mp
from time import sleep
from random import random, randint
from sys import stdout

In [10]:
class Taquero(mp.Process):
    def __init__(self, id, cupo_canasta, tacos_canasta, mutex, canasta):
        mp.Process.__init__(self)
        self.id = str(id)
        self.cupo_canasta = cupo_canasta
        self.tacos_canasta = tacos_canasta
        self.mutex = mutex
        self.canasta = canasta

    def trabajar(self):
        sleep(0.1)
        self.cupo_canasta.acquire()
        self.mutex.acquire()
        taco = randint(1,100)
        self.canasta.put(taco)
        stdout.write(">> T" + self.id + ": pongo taco: " + str(taco) + " Número de tacos: " + str(self.canasta.qsize()) + "\n")
        self.mutex.release()
        self.tacos_canasta.release()
        
    def run(self):
        stdout.write("Taquero con ID: " + self.id + "\n")
        while True:
            self.trabajar()

        
class Cliente(mp.Process):
    def __init__(self, id, cupo_canasta, tacos_canasta, mutex, canasta, hambre):
        mp.Process.__init__(self)
        self.id = str(id)
        self.cupo_canasta = cupo_canasta
        self.tacos_canasta = tacos_canasta
        self.mutex = mutex
        self.canasta = canasta
        
        self.hambre = hambre
        self.tacos_comidos = 0

    def comer(self):
        self.tacos_canasta.acquire()
        self.mutex.acquire()
        taco = self.canasta.get()
        stdout.write("<< C" + self.id + ": tomo taco: " + str(taco) + " Número de tacos: " + str(self.canasta.qsize()) + "\n")
        self.mutex.release()
        self.cupo_canasta.release()
        sleep(0.5)
        
    def run(self):
        stdout.write("Cliente ID: " + self.id + ". Tengo hambre de " + str(self.hambre) + " tacos\n")
        while self.tacos_comidos < self.hambre:
            self.comer()
            self.tacos_comidos += 1
        stdout.write("\nCliente " + self.id + ": Estoy satisfecho\n\n")


class Taqueria():
    def __init__(self, tacos, taqueros, clientes):
        self.n_taqueros = taqueros
        self.n_clientes = clientes
        
        self.canasta = mp.Queue()
        self.cupo_canasta = mp.Semaphore(tacos)
        self.tacos_canasta = mp.Semaphore(0)
        self.mutex = mp.Lock()
    
    def prepara(self):
        self.taqueros = [Taquero(i, self.cupo_canasta, self.tacos_canasta, self.mutex, self.canasta) for i in range(self.n_taqueros)]
        self.clientes = [Cliente(i, self.cupo_canasta, self.tacos_canasta, self.mutex, self.canasta, randint(5,10)) for i in range(self.n_clientes)]
    
    def abre(self):
        for i in range(self.n_taqueros):
            self.taqueros[i].start()
        for i in range(self.n_clientes):
            self.clientes[i].start()
    
def main(tacos, taqueros, clientes):
    taqueria = Taqueria(tacos, taqueros, clientes)
    taqueria.prepara()
    taqueria.abre()

In [11]:
tacos = 20
taqueros = 1
clientes = 1
main(tacos, taqueros, clientes)

Taquero con ID: 0
Cliente ID: 0. Tengo hambre de 10 tacos
>> T0: pongo taco: 82 Número de tacos: 1
<< C0: tomo taco: 82 Número de tacos: 0
>> T0: pongo taco: 34 Número de tacos: 1
>> T0: pongo taco: 36 Número de tacos: 2
>> T0: pongo taco: 47 Número de tacos: 3
>> T0: pongo taco: 94 Número de tacos: 4
<< C0: tomo taco: 34 Número de tacos: 3
>> T0: pongo taco: 43 Número de tacos: 4
>> T0: pongo taco: 39 Número de tacos: 5
>> T0: pongo taco: 19 Número de tacos: 6
>> T0: pongo taco: 5 Número de tacos: 7
>> T0: pongo taco: 39 Número de tacos: 8
<< C0: tomo taco: 36 Número de tacos: 7
>> T0: pongo taco: 35 Número de tacos: 8
>> T0: pongo taco: 50 Número de tacos: 9
>> T0: pongo taco: 21 Número de tacos: 10
>> T0: pongo taco: 16 Número de tacos: 11
>> T0: pongo taco: 77 Número de tacos: 12
<< C0: tomo taco: 47 Número de tacos: 11
>> T0: pongo taco: 91 Número de tacos: 12
>> T0: pongo taco: 56 Número de tacos: 13
>> T0: pongo taco: 43 Número de tacos: 14
>> T0: pongo taco: 52 Número de tacos:

Process Cliente-22:
Process Taquero-21:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/bondrewd/miniconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/bondrewd/miniconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "<ipython-input-10-83d905c99c02>", line 50, in run
    self.comer()
  File "<ipython-input-10-83d905c99c02>", line 23, in run
    self.trabajar()
  File "<ipython-input-10-83d905c99c02>", line 45, in comer
    sleep(0.5)
KeyboardInterrupt
  File "<ipython-input-10-83d905c99c02>", line 11, in trabajar
    sleep(0.1)
KeyboardInterrupt
